In [1]:
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import PlayByPlayV3, ScoreboardV2, LeagueGameLog, LeagueStandingsV3, HustleStatsBoxScore, BoxScoreTraditionalV2, LeagueDashPlayerStats
from nba_api.stats.static import teams

# Play by Play

Load CSV

In [3]:
file = '/Users/brandonbarber/Desktop/DS340W Project/Model/Raw Data Files/pbp_2024_25_regularseason/0022400061.csv'
pbp = pd.read_csv(file)


Metadata

In [4]:
pbp.info()
pbp.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   GAME_ID                    388 non-null    int64  
 1   EVENTNUM                   388 non-null    int64  
 2   EVENTMSGTYPE               388 non-null    int64  
 3   EVENTMSGACTIONTYPE         388 non-null    int64  
 4   PERIOD                     388 non-null    int64  
 5   WCTIMESTRING               388 non-null    object 
 6   PCTIMESTRING               388 non-null    object 
 7   HOMEDESCRIPTION            202 non-null    object 
 8   NEUTRALDESCRIPTION         9 non-null      object 
 9   VISITORDESCRIPTION         191 non-null    object 
 10  SCORE                      118 non-null    object 
 11  SCOREMARGIN                118 non-null    object 
 12  PERSON1TYPE                388 non-null    int64  
 13  PLAYER1_ID                 388 non-null    int64  

(388, 34)

Column Selection

In [5]:
pbp = pbp.loc[:, ['GAME_ID', 'PERIOD', 'PCTIMESTRING', 'SCORE', 'SCOREMARGIN', 'HOMEDESCRIPTION', 'VISITORDESCRIPTION']]

In [6]:
pbp

,GAME_ID,PERIOD,PCTIMESTRING,SCORE,SCOREMARGIN,HOMEDESCRIPTION,VISITORDESCRIPTION
0,22400061,1,12:00,NaN,NaN,NaN,NaN
1,22400061,1,12:00,NaN,NaN,Jump Ball Horford vs. Towns: Tip to White,NaN
2,22400061,1,11:48,0 - 3,3,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN
3,22400061,1,11:27,2 - 3,1,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST)
4,22400061,1,11:16,NaN,NaN,MISS Tatum 26' 3PT Pullup Jump Shot,NaN
...,...,...,...,...,...,...,...
383,22400061,4,0:51,NaN,NaN,NaN,Knicks Rebound
384,22400061,4,0:36,NaN,NaN,NaN,MISS Toppin 25' 3PT Jump Shot
385,22400061,4,0:35,NaN,NaN,CELTICS Rebound,NaN
386,22400061,4,0:11,NaN,NaN,CELTICS Turnover: Shot Clock (T#4),NaN


Score Margin and Period as numbers

In [7]:
pbp["SCOREMARGIN"] = pd.to_numeric(pbp["SCOREMARGIN"], errors="coerce")
pbp["PERIOD"] = pd.to_numeric(pbp["PERIOD"], errors="coerce")

Translate Time into seconds left in game

In [8]:

mins_secs = pbp['PCTIMESTRING'].str.split(':', expand=True).astype('int')
sec_in_period = mins_secs[0] * 60 + mins_secs[1]
pbp["SECONDS REMAINING"] = sec_in_period + (720 * (4 - pbp["PERIOD"]))

Remove NA from score margin and score

In [9]:
scores = pbp["SCORE"].str.split("-", expand=True)

# convert to numeric (handles NaN safely)
away = pd.to_numeric(scores[0].str.strip(), errors="coerce")
home = pd.to_numeric(scores[1].str.strip(), errors="coerce")

# --- 2) Find tie rows (away == home) ---
tie_mask = (away == home)

# --- 3) For tie rows where SCOREMARGIN is NaN, set SCOREMARGIN = 0 ---
pbp.loc[tie_mask & pbp["SCOREMARGIN"].isna(), "SCOREMARGIN"] = 0

pbp["HOME_SCORE"] = home
pbp["VISITOR_SCORE"] = away
#Replaces all NA after a scoring play with the previous score margin and score

pbp["SCOREMARGIN"] = pbp["SCOREMARGIN"].ffill().fillna(0)
pbp["SCORE"] = pbp["SCORE"].ffill().fillna(0)
pbp["HOME_SCORE"] = pbp["HOME_SCORE"].ffill().fillna(0)
pbp["VISITOR_SCORE"] = pbp["VISITOR_SCORE"].ffill().fillna(0)





In [10]:
pbp.tail(5)

,GAME_ID,PERIOD,PCTIMESTRING,SCORE,SCOREMARGIN,HOMEDESCRIPTION,VISITORDESCRIPTION,SECONDS REMAINING,HOME_SCORE,VISITOR_SCORE
383,22400061,4,0:51,109 - 132,23.0,NaN,Knicks Rebound,51,132.0,109.0
384,22400061,4,0:36,109 - 132,23.0,NaN,MISS Toppin 25' 3PT Jump Shot,36,132.0,109.0
385,22400061,4,0:35,109 - 132,23.0,CELTICS Rebound,NaN,35,132.0,109.0
386,22400061,4,0:11,109 - 132,23.0,CELTICS Turnover: Shot Clock (T#4),NaN,11,132.0,109.0
387,22400061,4,0:00,109 - 132,23.0,NaN,NaN,0,132.0,109.0


Add label for Winner (Home = 1, Away - 0)

In [11]:
if pbp.iloc[-1]["HOME_SCORE"] > pbp.iloc[-1]["VISITOR_SCORE"]:
    pbp["WINNER"] = 1
else: 
    pbp["WINNER"] = 0


In [12]:
pbp = pbp.loc[:, ["GAME_ID", "SECONDS REMAINING", "HOME_SCORE", "VISITOR_SCORE", "SCOREMARGIN", "HOMEDESCRIPTION", "VISITORDESCRIPTION", "WINNER"]]
pbp.head(25)

,GAME_ID,SECONDS REMAINING,HOME_SCORE,VISITOR_SCORE,SCOREMARGIN,HOMEDESCRIPTION,VISITORDESCRIPTION,WINNER
0,22400061,2880,0.0,0.0,0.0,NaN,NaN,1
1,22400061,2880,0.0,0.0,0.0,Jump Ball Horford vs. Towns: Tip to White,NaN,1
2,22400061,2868,3.0,0.0,3.0,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN,1
3,22400061,2847,3.0,2.0,1.0,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),1
4,22400061,2836,3.0,2.0,1.0,MISS Tatum 26' 3PT Pullup Jump Shot,NaN,1
5,22400061,2835,3.0,2.0,1.0,NaN,Anunoby REBOUND (Off:0 Def:1),1
6,22400061,2829,3.0,4.0,-1.0,NaN,Brunson 3' Running Layup (2 PTS),1
7,22400061,2814,5.0,4.0,1.0,White 5' Driving Floating Jump Shot (2 PTS) (T...,NaN,1
8,22400061,2795,5.0,6.0,-1.0,NaN,Brunson 13' Pullup Jump Shot (4 PTS),1
9,22400061,2781,8.0,6.0,2.0,Horford 25' 3PT Jump Shot (3 PTS) (Tatum 2 AST),NaN,1


# Scoreboard

In [13]:
scoreboard = ScoreboardV2(game_date="10/22/2024")  # mm/dd/yyyy
sb = scoreboard.get_data_frames()[0]   # "GameHeader"
sb

,GAME_DATE_EST,GAME_SEQUENCE,GAME_ID,GAME_STATUS_ID,GAME_STATUS_TEXT,GAMECODE,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,LIVE_PERIOD,LIVE_PC_TIME,NATL_TV_BROADCASTER_ABBREVIATION,HOME_TV_BROADCASTER_ABBREVIATION,AWAY_TV_BROADCASTER_ABBREVIATION,LIVE_PERIOD_TIME_BCAST,ARENA_NAME,WH_STATUS,WNBA_COMMISSIONER_FLAG
0,2024-10-22T00:00:00,1,0022400061,3,Final,20241022/NYKBOS,1610612738,1610612752,2024,4,,TNT/Max,None,None,Q4 - TNT/Max,TD Garden,1,0
1,2024-10-22T00:00:00,2,0022400062,3,Final,20241022/MINLAL,1610612747,1610612750,2024,4,,TNT/Max,None,None,Q4 - TNT/Max,Crypto.com Arena,1,0


In [34]:
box = BoxScoreTraditionalV2(game_id='0022400061')
box_dfs = box.get_data_frames()
lineup = box_dfs[0] 
lineup

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022400061,1610612752,NYK,New York,1628384,OG Anunoby,OG,F,,34:10,...,0.0,5.0,5.0,3.0,1.0,0.0,0.0,3.0,4.0,-21.0
1,0022400061,1610612752,NYK,New York,1628404,Josh Hart,Josh,F,,24:30,...,1.0,3.0,4.0,3.0,0.0,0.0,1.0,0.0,12.0,-23.0
2,0022400061,1610612752,NYK,New York,1626157,Karl-Anthony Towns,Karl-Anthony,C,,23:37,...,0.0,7.0,7.0,3.0,0.0,0.0,0.0,1.0,12.0,-18.0
3,0022400061,1610612752,NYK,New York,1628969,Mikal Bridges,Mikal,G,,34:37,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,16.0,-33.0
4,0022400061,1610612752,NYK,New York,1628973,Jalen Brunson,Jalen,G,,24:30,...,0.0,1.0,1.0,2.0,0.0,0.0,4.0,3.0,22.0,-23.0
5,0022400061,1610612752,NYK,New York,1630579,Jericho Sims,Jericho,,,24:23,...,3.0,6.0,9.0,1.0,0.0,2.0,1.0,0.0,4.0,-5.0
6,0022400061,1610612752,NYK,New York,1630540,Miles McBride,Miles,,,25:51,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,22.0,-18.0
7,0022400061,1610612752,NYK,New York,1626166,Cameron Payne,Cameron,,,20:43,...,1.0,2.0,3.0,4.0,1.0,0.0,1.0,3.0,11.0,0.0
8,0022400061,1610612752,NYK,New York,1642359,Pacôme Dadiet,Pacôme,,,13:23,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,10.0
9,0022400061,1610612752,NYK,New York,1630574,Ariel Hukporti,Ariel,,,6:05,...,0.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0


In [35]:
game_log = LeagueGameLog(
    season= "2024-25",
    season_type_all_star='Regular Season'
)
game_log_df = game_log.get_data_frames()[0]
game_date  = game_log_df.loc[game_log_df['GAME_ID'] == '0022400061', 'GAME_DATE'].iloc[0]

In [36]:
game_date

'2024-10-22'

Get columns

In [14]:
sb = sb.loc[ :, ["GAME_DATE_EST", "GAME_ID", "HOME_TEAM_ID", "VISITOR_TEAM_ID"]]
sb

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID
0,2024-10-22T00:00:00,0022400061,1610612738,1610612752
1,2024-10-22T00:00:00,0022400062,1610612747,1610612750


Clean Date

In [15]:
sb["GAME_DATE_EST"] = sb["GAME_DATE_EST"].str[0:10]
sb["GAME_ID"] = sb["GAME_ID"].str[2:]
sb

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID
0,2024-10-22,22400061,1610612738,1610612752
1,2024-10-22,22400062,1610612747,1610612750


Clean GAME_ID

In [16]:

sb["GAME_ID"] = pd.to_numeric(sb["GAME_ID"], errors="coerce")
sb

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID
0,2024-10-22,22400061,1610612738,1610612752
1,2024-10-22,22400062,1610612747,1610612750


# Teams

In [17]:
all_teams = teams.get_teams()
teams_df = pd.DataFrame(all_teams)
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


Get columns

In [18]:
teams_df = teams_df.loc[:, ["id", "abbreviation"]]
teams_df.head()

,id,abbreviation
0,1610612737,ATL
1,1610612738,BOS
2,1610612739,CLE
3,1610612740,NOP
4,1610612741,CHI


# Elo

In [20]:
file2 = '/Users/brandonbarber/Desktop/DS340W Project/Model/Raw Data Files/nba_elo.csv'
nba_elo_df = pd.read_csv(file2)

nba_elo_df = nba_elo_df.loc[nba_elo_df["date"] == '2024-10-22']

nba_elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2,is_home
149364,2024-10-22,2025,0,NaN,BOS,NYK,1690.516906,1564.085757,0.786414,0.213586,1697.051482,1557.551181,132,109,1
149365,2024-10-22,2025,0,NaN,NYK,BOS,1564.085757,1690.516906,0.213586,0.786414,1557.551181,1697.051482,109,132,0
149366,2024-10-22,2025,0,NaN,LAL,MIN,1565.246192,1625.451107,0.557021,0.442979,1572.469590,1618.227708,110,103,1
149367,2024-10-22,2025,0,NaN,MIN,LAL,1625.451107,1565.246192,0.442979,0.557021,1618.227708,1572.469590,103,110,0


Get Columns

In [21]:
nba_elo_df = nba_elo_df.loc[:, ["date", "team1", "team2", "elo1_pre", "elo2_pre"]]
nba_elo_df

,date,team1,team2,elo1_pre,elo2_pre
149364,2024-10-22,BOS,NYK,1690.516906,1564.085757
149365,2024-10-22,NYK,BOS,1564.085757,1690.516906
149366,2024-10-22,LAL,MIN,1565.246192,1625.451107
149367,2024-10-22,MIN,LAL,1625.451107,1565.246192


Elo difference

In [22]:
nba_elo_df["ELO_DIFF"] = (nba_elo_df['elo1_pre'] - nba_elo_df['elo2_pre'])

# Lineup

In [23]:
box = BoxScoreTraditionalV2(game_id= "0022400061")
lineup = box.get_data_frames()[0]
lineup

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022400061,1610612752,NYK,New York,1628384,OG Anunoby,OG,F,,34:10,...,0.0,5.0,5.0,3.0,1.0,0.0,0.0,3.0,4.0,-21.0
1,0022400061,1610612752,NYK,New York,1628404,Josh Hart,Josh,F,,24:30,...,1.0,3.0,4.0,3.0,0.0,0.0,1.0,0.0,12.0,-23.0
2,0022400061,1610612752,NYK,New York,1626157,Karl-Anthony Towns,Karl-Anthony,C,,23:37,...,0.0,7.0,7.0,3.0,0.0,0.0,0.0,1.0,12.0,-18.0
3,0022400061,1610612752,NYK,New York,1628969,Mikal Bridges,Mikal,G,,34:37,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,16.0,-33.0
4,0022400061,1610612752,NYK,New York,1628973,Jalen Brunson,Jalen,G,,24:30,...,0.0,1.0,1.0,2.0,0.0,0.0,4.0,3.0,22.0,-23.0
5,0022400061,1610612752,NYK,New York,1630579,Jericho Sims,Jericho,,,24:23,...,3.0,6.0,9.0,1.0,0.0,2.0,1.0,0.0,4.0,-5.0
6,0022400061,1610612752,NYK,New York,1630540,Miles McBride,Miles,,,25:51,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,22.0,-18.0
7,0022400061,1610612752,NYK,New York,1626166,Cameron Payne,Cameron,,,20:43,...,1.0,2.0,3.0,4.0,1.0,0.0,1.0,3.0,11.0,0.0
8,0022400061,1610612752,NYK,New York,1642359,Pacôme Dadiet,Pacôme,,,13:23,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,10.0
9,0022400061,1610612752,NYK,New York,1630574,Ariel Hukporti,Ariel,,,6:05,...,0.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,9.0


Remove DNP

In [ ]:
lineup = lineup[~lineup["COMMENT"].str.contains("DNP", case=False, na=False)]
lineup

Select Columns

In [ ]:
lineup = lineup[["GAME_ID", "TEAM_ID", "TEAM_ABBREVIATION", "PLAYER_ID", "PLAYER_NAME", "START_POSITION" ]]
lineup


Split Name

In [ ]:
lineup = lineup.copy()  # safe, no warnings
lineup[["FIRST_NAME", "LAST_NAME"]] = lineup["PLAYER_NAME"].str.split(" ", n=1, expand=True)
lineup.head(1)


Reorder Columns

In [ ]:
lineup = lineup.iloc[:, [0,1,3,4,6,7,2,5]]


IDs all numerical

In [ ]:
lineup["GAME_ID"] = pd.to_numeric(lineup["GAME_ID"], errors="coerce")
lineup["TEAM_ID"] = pd.to_numeric(lineup["TEAM_ID"], errors="coerce")
lineup["TEAM_ID"] = pd.to_numeric(lineup["TEAM_ID"], errors="coerce")

# Player Stats

In [ ]:
dash = LeagueDashPlayerStats(
    season= "2024-25",
    season_type_all_star= "Regular Season",       # "Regular Season" / "Playoffs"
    per_mode_detailed="PerGame",
    date_to_nullable="10/22/2024"                # EXCLUDES the game day itself
)
plyr_avgs = dash.get_data_frames()[0]

In [ ]:
plyr_avgs = plyr_avgs[["PLAYER_ID", 'PTS', "REB", "AST", "PLUS_MINUS"]]
plyr_avgs

# Joins

### Scoreboard w/ Teams

In [ ]:
#Home Team Merge
sb = sb.merge(
    teams_df,
    left_on = "HOME_TEAM_ID",
    right_on = "id",
    how = 'left'
).rename(columns={"abbreviation": "HOME_TEAM"}).drop(columns="id")

#Away Team Merge
sb = sb.merge(
    teams_df,
    left_on = "VISITOR_TEAM_ID",
    right_on = "id",
    how = 'left'
).rename(columns={"abbreviation": "VISITOR_TEAM"}).drop(columns="id")

In [ ]:
sb

### Scoreboard w/ ELO

In [ ]:
sb = sb.merge(
    nba_elo_df,
    left_on = ["GAME_DATE_EST", "HOME_TEAM", "VISITOR_TEAM"],
    right_on = ["date", "team1", "team2"],
    how = "left"
).rename(columns={"elo1_pre": "HOME_ELO", "elo2_pre": "VISITOR_ELO"}).drop(columns=["date", "team1", "team2"])

In [ ]:
sb

### PBP w/ Scoreboard

In [ ]:
pbp = pbp.merge(
    sb,
    left_on= "GAME_ID",
    right_on= "GAME_ID",
    how= 'left'
)

Remove rows where game state doesn't change

In [ ]:
pbp = pbp[~(pbp["HOMEDESCRIPTION"].isna() & pbp["VISITORDESCRIPTION"].isna())]
pbp.head()

Reorder

In [ ]:
pbp = pbp.iloc[: , [0,8,9,11,10,12,13,14,15,1,2,3,4,5,6,7]]
pbp.head(50)

### Lineups w/ Player Stats

In [ ]:
lineup = lineup.merge(
    plyr_avgs,
    left_on = "PLAYER_ID",
    right_on = "PLAYER_ID",
    how = 'left'
)

In [ ]:
lineup_starters = lineup.loc[lineup["START_POSITION"] != '']

In [ ]:
lineup

# Lineup Updates for PBP

Initialize Starters/ Update Subs

In [ ]:
import re
import numpy as np
import pandas as pd

# ------------------------------------------------------------
# Inputs (single game):
#   pbp   : DataFrame with columns (UPPERCASE): 
#           GAME_ID, HOME_TEAM_ID, VISITOR_TEAM_ID, HOMEDESCRIPTION, VISITORDESCRIPTION
#   lineup: DataFrame for this game: 
#           GAME_ID, TEAM_ID, PLAYER_ID, FIRST_NAME, LAST_NAME, START_POSITION, 
#           PTS, REB, AST, PLUS_MINUS   (stats can be season or game — your choice)
# Output:
#   test  : a copy of pbp with HOME_/VISITOR_ on-court player + stats columns per row
# ------------------------------------------------------------

# ---------- helpers ----------
_POS_ORDER = {'G': 0, 'F': 1, 'C': 2}

def order_starters(df: pd.DataFrame) -> pd.DataFrame:
    """Order starters G < F < C then LAST_NAME for stable slotting."""
    return (
        df.assign(_pos=df['START_POSITION'].map(_POS_ORDER).fillna(9))
          .sort_values(['_pos','LAST_NAME','FIRST_NAME'])
    )

def starters_for_team(lineup: pd.DataFrame, team_id) -> pd.DataFrame:
    return order_starters(lineup[(lineup['TEAM_ID'] == team_id) & lineup['START_POSITION'].notna()])

def stat_lookup(lineup: pd.DataFrame):
    """
    Build lookups by:
      - exact (TEAM_ID, FIRST_NAME, LAST_NAME)
      - fallback (TEAM_ID, LAST_NAME)
    """
    lu_exact, lu_last = {}, {}
    # normalize strings once
    L = lineup.copy()
    for c in ['FIRST_NAME','LAST_NAME']:
        L[c] = L[c].astype(str).str.strip()
    for r in L.itertuples(index=False):
        key_exact = (r.TEAM_ID, r.FIRST_NAME, r.LAST_NAME)
        key_last  = (r.TEAM_ID, r.LAST_NAME)
        payload = {
            'PLAYER_ID': int(r.PLAYER_ID) if pd.notna(r.PLAYER_ID) else None,
            'PPG': r.PTS, 'APG': r.AST, 'RPG': r.REB, 'PLUSMIN': r.PLUS_MINUS
        }
        lu_exact[key_exact] = payload
        # prefer exact later, so only set last-name fallback if not set
        lu_last.setdefault(key_last, payload)
    return lu_exact, lu_last

_sub_re = re.compile(r"SUB:\s*(.*?)\s+FOR\s+(.*)", flags=re.IGNORECASE)

def _split_name(txt: str):
    """Return (first,last) if possible; else (None,last) with last token as last name."""
    if not isinstance(txt, str):
        return (None, None)
    t = re.sub(r'[\(\)\.,;:!\?]', ' ', txt)     # strip punctuation
    t = re.sub(r'\s+', ' ', t).strip()
    if not t:
        return (None, None)
    parts = t.split(' ')
    if len(parts) == 1:
        return (None, parts[0])
    return (parts[0], parts[-1])

def parse_sub_line(text):
    """
    From 'SUB: Payton Pritchard FOR Jaylen Brown'
    return ((in_first,in_last), (out_first,out_last))
    or (None, None) if no sub.
    """
    if not isinstance(text, str) or 'SUB' not in text:
        return (None, None)
    m = _sub_re.search(text)
    if not m:
        return (None, None)
    in_raw, out_raw = m.group(1).strip(), m.group(2).strip()
    return _split_name(in_raw), _split_name(out_raw)

def find_slot(current_five, target_first, target_last):
    """
    Find the index in current_five matching (first,last) primarily,
    else by last name, else return None.
    current_five is a list of dicts: {'FIRST_NAME','LAST_NAME',...}
    """
    # exact first+last
    for i, p in enumerate(current_five):
        if (p['FIRST_NAME'] and target_first and p['FIRST_NAME'].lower() == str(target_first).lower()
            and p['LAST_NAME'] and target_last and p['LAST_NAME'].lower() == str(target_last).lower()):
            return i
    # fallback: last name only
    for i, p in enumerate(current_five):
        if p['LAST_NAME'] and target_last and p['LAST_NAME'].lower() == str(target_last).lower():
            return i
    return None

def player_payload(lineup_team_df: pd.DataFrame, first, last):
    """
    Return the canonical payload dict for a player on a given team:
      {'FIRST_NAME','LAST_NAME','PLAYER_ID','PPG','APG','RPG','PLUSMIN'}
    or None if not found.
    """
    if last is None:
        return None
    # quick filters reduce the scan
    cand = lineup_team_df[lineup_team_df['LAST_NAME'].str.lower() == str(last).lower()]
    if first:
        cand2 = cand[cand['FIRST_NAME'].str.lower() == str(first).lower()]
        if not cand2.empty:
            row = cand2.iloc[0]
        elif not cand.empty:
            row = cand.iloc[0]
        else:
            return None
    else:
        if cand.empty:
            return None
        row = cand.iloc[0]
    return {
        'FIRST_NAME': row['FIRST_NAME'],
        'LAST_NAME' : row['LAST_NAME'],
        'PLAYER_ID' : int(row['PLAYER_ID']) if pd.notna(row['PLAYER_ID']) else None,
        'PPG'      : row['PTS'],
        'APG'      : row['AST'],
        'RPG'      : row['REB'],
        'PLUSMIN'  : row['PLUS_MINUS'],
    }

def starters_payloads(lineup_team_df: pd.DataFrame):
    """
    Return list of 5 payload dicts for starters on this team.
    Payload fields: FIRST_NAME, LAST_NAME, PLAYER_ID, PPG, APG, RPG, PLUSMIN
    """
    st = starters_for_team(lineup_team_df, lineup_team_df['TEAM_ID'].iat[0]) if 'TEAM_ID' in lineup_team_df else lineup_team_df
    st = order_starters(lineup_team_df)
    rows = st[['FIRST_NAME','LAST_NAME','PLAYER_ID','PTS','AST','REB','PLUS_MINUS']].to_dict('records')
    # pad to 5 if needed
    while len(rows) < 5:
        rows.append({'FIRST_NAME': None, 'LAST_NAME': None, 'PLAYER_ID': None, 'PTS': None, 'AST': None, 'REB': None, 'PLUS_MINUS': None})
    out = []
    for r in rows[:5]:
        out.append({
            'FIRST_NAME': r['FIRST_NAME'],
            'LAST_NAME' : r['LAST_NAME'],
            'PLAYER_ID' : int(r['PLAYER_ID']) if pd.notna(r['PLAYER_ID']) else None,
            'PPG'      : r['PTS'],
            'APG'      : r['AST'],
            'RPG'      : r['REB'],
            'PLUSMIN'  : r['PLUS_MINUS'],
        })
    return out

def write_side_cols(d, side_prefix, five_payloads):
    """Write player+stat columns for one side into dict d (for a single row)."""
    for i, p in enumerate(five_payloads[:5]):
        d[f'{side_prefix}_PLAYER_{i}'] = p['LAST_NAME']
        d[f'{side_prefix}_PLAYER_{i}_ID'] = (np.int64(p['PLAYER_ID']) if p['PLAYER_ID'] is not None else None)
        d[f'{side_prefix}_PLAYER_{i}_PPG'] = p['PPG']
        d[f'{side_prefix}_PLAYER_{i}_APG'] = p['APG']
        d[f'{side_prefix}_PLAYER_{i}_RPG'] = p['RPG']
        d[f'{side_prefix}_PLAYER_{i}_PLUSMIN'] = p['PLUSMIN']

# ---------- main (single-game) ----------
def build_on_court_with_subs_single_game(pbp: pd.DataFrame, lineup: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a copy of pbp with HOME_/VISITOR_ on-court columns updated per row
    based on substitutions found in HOMEDESCRIPTION / VISITORDESCRIPTION.
    """
    df = pbp.copy()

    home_id    = df['HOME_TEAM_ID'].iat[0]
    visitor_id = df['VISITOR_TEAM_ID'].iat[0]

    # split lineup by team + normalize names
    L = lineup.copy()
    for c in ['FIRST_NAME','LAST_NAME']:
        L[c] = L[c].astype(str).str.strip()
    L_home  = L[L['TEAM_ID'] == home_id].reset_index(drop=True)
    L_visit = L[L['TEAM_ID'] == visitor_id].reset_index(drop=True)

    # initial on-court (starters)
    home_on  = starters_payloads(L_home)
    visit_on = starters_payloads(L_visit)

    out_rows = []
    for idx, row in df.iterrows():
        row_out = {}

        # Home substitution?
        in_h, out_h = parse_sub_line(row.get('HOMEDESCRIPTION'))
        if in_h and out_h:
            slot = find_slot(home_on, *out_h)
            if slot is not None:
                payload = player_payload(L_home, *in_h)
                if payload:
                    home_on[slot] = payload  # replace in-place

        # Visitor substitution?
        in_v, out_v = parse_sub_line(row.get('VISITORDESCRIPTION'))
        if in_v and out_v:
            slot = find_slot(visit_on, *out_v)
            if slot is not None:
                payload = player_payload(L_visit, *in_v)
                if payload:
                    visit_on[slot] = payload

        # write current state for this play
        write_side_cols(row_out, 'HOME',    home_on)
        write_side_cols(row_out, 'VISITOR', visit_on)
        out_rows.append(row_out)

    wide = pd.DataFrame(out_rows, index=df.index)
    return pd.concat([df, wide], axis=1)

# ----------------- run (pbp unchanged) -----------------
pd.set_option('display.max_columns', None)
pbp = build_on_court_with_subs_single_game(pbp, lineup)


In [ ]:
pd.set_option('display.max_columns', None)
pbp.head(15)


Team Stat Calculation

In [ ]:
pbp["HOME_PPG_TOTAL"] = pbp.filter(regex=r"^HOME_PLAYER_\d+_PPG$").sum(axis=1)
pbp["HOME_APG_TOTAL"] = pbp.filter(regex=r"^HOME_PLAYER_\d+_APG$").sum(axis=1)
pbp["HOME_RPG_TOTAL"] = pbp.filter(regex=r"^HOME_PLAYER_\d+_RPG$").sum(axis=1)
pbp["HOME_PLUSMIN_TOTAL"] = pbp.filter(regex=r"^HOME_PLAYER_\d+_PLUSMIN$").sum(axis=1)

In [ ]:
pbp["VISITOR_PPG_TOTAL"] = pbp.filter(regex=r"^VISITOR_PLAYER_\d+_PPG$").sum(axis=1)
pbp["VISITOR_APG_TOTAL"] = pbp.filter(regex=r"^VISITOR_PLAYER_\d+_APG$").sum(axis=1)
pbp["VISITOR_RPG_TOTAL"] = pbp.filter(regex=r"^VISITOR_PLAYER_\d+_RPG$").sum(axis=1)
pbp["VISITOR_PLUSMIN_TOTAL"] = pbp.filter(regex=r"^VISITOR_PLAYER_\d+_PLUSMIN$").sum(axis=1)

# Final Table

In [50]:
pbp

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,HOME_TEAM,VISITOR_TEAM_ID,VISITOR_TEAM,HOME_ELO,VISITOR_ELO,ELO_DIFF,SECONDS REMAINING,HOME_SCORE,VISITOR_SCORE,SCOREMARGIN,HOMEDESCRIPTION,VISITORDESCRIPTION,WINNER,HOME_PLAYER_0,HOME_PLAYER_0_ID,HOME_PLAYER_0_PPG,HOME_PLAYER_0_APG,HOME_PLAYER_0_RPG,HOME_PLAYER_0_PLUSMIN,HOME_PLAYER_1,HOME_PLAYER_1_ID,HOME_PLAYER_1_PPG,HOME_PLAYER_1_APG,HOME_PLAYER_1_RPG,HOME_PLAYER_1_PLUSMIN,HOME_PLAYER_2,HOME_PLAYER_2_ID,HOME_PLAYER_2_PPG,HOME_PLAYER_2_APG,HOME_PLAYER_2_RPG,HOME_PLAYER_2_PLUSMIN,HOME_PLAYER_3,HOME_PLAYER_3_ID,HOME_PLAYER_3_PPG,HOME_PLAYER_3_APG,HOME_PLAYER_3_RPG,HOME_PLAYER_3_PLUSMIN,HOME_PLAYER_4,HOME_PLAYER_4_ID,HOME_PLAYER_4_PPG,HOME_PLAYER_4_APG,HOME_PLAYER_4_RPG,HOME_PLAYER_4_PLUSMIN,VISITOR_PLAYER_0,VISITOR_PLAYER_0_ID,VISITOR_PLAYER_0_PPG,VISITOR_PLAYER_0_APG,VISITOR_PLAYER_0_RPG,VISITOR_PLAYER_0_PLUSMIN,VISITOR_PLAYER_1,VISITOR_PLAYER_1_ID,VISITOR_PLAYER_1_PPG,VISITOR_PLAYER_1_APG,VISITOR_PLAYER_1_RPG,VISITOR_PLAYER_1_PLUSMIN,VISITOR_PLAYER_2,VISITOR_PLAYER_2_ID,VISITOR_PLAYER_2_PPG,VISITOR_PLAYER_2_APG,VISITOR_PLAYER_2_RPG,VISITOR_PLAYER_2_PLUSMIN,VISITOR_PLAYER_3,VISITOR_PLAYER_3_ID,VISITOR_PLAYER_3_PPG,VISITOR_PLAYER_3_APG,VISITOR_PLAYER_3_RPG,VISITOR_PLAYER_3_PLUSMIN,VISITOR_PLAYER_4,VISITOR_PLAYER_4_ID,VISITOR_PLAYER_4_PPG,VISITOR_PLAYER_4_APG,VISITOR_PLAYER_4_RPG,VISITOR_PLAYER_4_PLUSMIN,HOME_PPG_TOTAL,HOME_APG_TOTAL,HOME_RPG_TOTAL,HOME_PLUSMIN_TOTAL,VISITOR_PPG_TOTAL,VISITOR_APG_TOTAL,VISITOR_RPG_TOTAL,VISITOR_PLUSMIN_TOTAL
1,22400061,2024-10-22,1610612738,BOS,1610612752,NYK,1690.516906,1564.085757,126.431149,2880,0.0,0.0,0.0,Jump Ball Horford vs. Towns: Tip to White,NaN,1,Holiday,201950,18.0,4.0,4.0,23.0,White,1628401,24.0,4.0,3.0,21.0,Brown,1627759,23.0,1.0,7.0,23.0,Tatum,1628369,37.0,10.0,4.0,26.0,Horford,201143,11.0,5.0,3.0,19.0,Bridges,1628969,16.0,2.0,0.0,-33.0,Brunson,1628973,22.0,2.0,1.0,-23.0,Anunoby,1628384,4.0,3.0,5.0,-21.0,Hart,1628404,12.0,3.0,4.0,-23.0,Towns,1626157,12.0,3.0,7.0,-18.0,113.0,24.0,21.0,112.0,66.0,13.0,17.0,-118.0
2,22400061,2024-10-22,1610612738,BOS,1610612752,NYK,1690.516906,1564.085757,126.431149,2868,3.0,0.0,3.0,Tatum 27' 3PT Pullup Jump Shot (3 PTS) (White ...,NaN,1,Holiday,201950,18.0,4.0,4.0,23.0,White,1628401,24.0,4.0,3.0,21.0,Brown,1627759,23.0,1.0,7.0,23.0,Tatum,1628369,37.0,10.0,4.0,26.0,Horford,201143,11.0,5.0,3.0,19.0,Bridges,1628969,16.0,2.0,0.0,-33.0,Brunson,1628973,22.0,2.0,1.0,-23.0,Anunoby,1628384,4.0,3.0,5.0,-21.0,Hart,1628404,12.0,3.0,4.0,-23.0,Towns,1626157,12.0,3.0,7.0,-18.0,113.0,24.0,21.0,112.0,66.0,13.0,17.0,-118.0
3,22400061,2024-10-22,1610612738,BOS,1610612752,NYK,1690.516906,1564.085757,126.431149,2847,3.0,2.0,1.0,NaN,Anunoby 6' Driving Layup (2 PTS) (Brunson 1 AST),1,Holiday,201950,18.0,4.0,4.0,23.0,White,1628401,24.0,4.0,3.0,21.0,Brown,1627759,23.0,1.0,7.0,23.0,Tatum,1628369,37.0,10.0,4.0,26.0,Horford,201143,11.0,5.0,3.0,19.0,Bridges,1628969,16.0,2.0,0.0,-33.0,Brunson,1628973,22.0,2.0,1.0,-23.0,Anunoby,1628384,4.0,3.0,5.0,-21.0,Hart,1628404,12.0,3.0,4.0,-23.0,Towns,1626157,12.0,3.0,7.0,-18.0,113.0,24.0,21.0,112.0,66.0,13.0,17.0,-118.0
4,22400061,2024-10-22,1610612738,BOS,1610612752,NYK,1690.516906,1564.085757,126.431149,2836,3.0,2.0,1.0,MISS Tatum 26' 3PT Pullup Jump Shot,NaN,1,Holiday,201950,18.0,4.0,4.0,23.0,White,1628401,24.0,4.0,3.0,21.0,Brown,1627759,23.0,1.0,7.0,23.0,Tatum,1628369,37.0,10.0,4.0,26.0,Horford,201143,11.0,5.0,3.0,19.0,Bridges,1628969,16.0,2.0,0.0,-33.0,Brunson,1628973,22.0,2.0,1.0,-23.0,Anunoby,1628384,4.0,3.0,5.0,-21.0,Hart,1628404,12.0,3.0,4.0,-23.0,Towns,1626157,12.0,3.0,7.0,-18.0,113.0,24.0,21.0,112.0,66.0,13.0,17.0,-118.0
5,22400061,2024-10-22,1610612738,BOS,1610612752,NYK,1690.516906,1564.085757,126.431149,2835,3.0,2.0,1.0,NaN,Anunoby REBOUND (Off:0 Def:1),1,Holiday,201950,18.0,4.0,4.0,23.0,White,1628401,24.0,4.0,3.0,21.0,Brown,1627759,23.0,1.0,7.0,23.0,Tatum,1628369,37.0,10.0,4.0,26.0,Horford,201143,11.0,5.0,3.0,19.0,Bridges,1628969,16.0,2.0,0.0,-33.0,Brunson,1628973,22.0,2.0,1.0,-23.0,Anun